# Introduction to Data Cleaning with Pandas

In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

### Sample dataset

Recall how to read a dataset from excel:

In [2]:
# Read in csv
grades = pd.read_csv('grades.csv')

In [3]:
# View the data
grades.head()

,assignment,weight,marks,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,1,0.1,93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0.1,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,0.1,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0.1,78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,0.1,92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Notice that there is a lot of columns with missing values. We need to clean the dataset first in order to make the data more useful.

Let's begin by setting our index to be the assignment numbers.

In [4]:
# Set index to assignment
grades.set_index('assignment', inplace=True)

## Data Cleaning

In [5]:
# Clean up the data by picking only the columns we need
grades = grades[['weight', 'marks']]

# View data
grades.head()

,weight,marks
assignment,,
1,0.1,93
2,0.1,44
3,0.1,75
4,0.1,78
5,0.1,92


Now, we want to determine the final grade of the student using these 10 assignments. First, we check to see if the weighting of these grades adds up to 100 to make sure we aren't missing any data.

## Summing over Rows/Columns

In [6]:
# Summing a column against all the rows (axis=0)
grades.weight.sum(axis=0)

0.8

This is less useful in our scenario here, but if you wanted to sum against all the columns, you would just set axis=1 instead of axis=0. 

In [7]:
# Summing a row against all the columns (axis=1)
grades.sum(axis=1)

assignment
1    93.1
2    44.1
3    75.1
4    78.1
5    92.1
6    90.1
7    95.1
8    57.1
dtype: float64

So we know that we are missing some data here. We happen to know that the remaining data is:
* Assignment 9: 78
* Assignment 10: 83. 

This data is stored in another dataframe called missing_data

## Appending Data

In [8]:
missing_data = pd.DataFrame({'assignment': [9,10],
                             'weight': [0.1, 0.1],
                             'marks': [78, 93]})

# Prepare data to append it to grades
missing_data.set_index('assignment', inplace=True)

missing_data.head()

,weight,marks
assignment,,
9,0.1,78
10,0.1,93


We use `df1.append(df2)` to add our missing data onto the end of our existing dataframe. This creates a new dataframe so we will call this `complete_grades`.

In [9]:
# Append missing data to grades
complete_grades = grades.append(missing_data)

complete_grades

,weight,marks
assignment,,
1,0.1,93
2,0.1,44
3,0.1,75
4,0.1,78
5,0.1,92
6,0.1,90
7,0.1,95
8,0.1,57
9,0.1,78


Since our index was already 9, 10 in the missing_data DataFrame, then we would not ignore the index. 

Next, let's calculate the weighting of each assignment from this student's grades. This would be multiplying the weight of each assignment by the marks and we'll store this in a new column, `mark_weight`.

## Multiplying Columns

In [10]:
# Multiply weight and marks 
complete_grades['mark_weight'] = complete_grades['weight'] * complete_grades['marks']

In [11]:
# View the data
complete_grades.head()

,weight,marks,mark_weight
assignment,,,
1,0.1,93,9.3
2,0.1,44,4.4
3,0.1,75,7.5
4,0.1,78,7.8
5,0.1,92,9.2


Adding, subtracting, multiplying and dividing columns does it element-by-element. 

Now, let's calculate the students final grade from these assignments.

In [12]:
# Calculate final grade
complete_grades['mark_weight'].sum(axis=0)

79.5

Now, what if you wanted to see the percent change in your grades over the assignments. There is a quick and easy way to do that! 

## Percentage Changes

In [13]:
# Percentage change between assignments
complete_grades.marks.pct_change()

assignment
1          NaN
2    -0.526882
3     0.704545
4     0.040000
5     0.179487
6    -0.021739
7     0.055556
8    -0.400000
9     0.368421
10    0.192308
Name: marks, dtype: float64

Note that it doesn't count the percentage change for the first assignment because there is no assignment 0 to reference to. We can see that for the most part the assignment grades are improving throughout the term. 

## Filtering

We went through some basic filtering with masks. There are a few other ways to filter your index or columns if they contain strings. For these purposes, we will use another set of data.

Suppose you make a diary of all the fruits you ate from Jan to March 2021. We want to do a quick analysis on these fruits to see what types of fruits you prefer.

In [14]:
# Import the data
fruits = pd.read_csv('fruits.csv')
fruits.set_index('Date', inplace=True)

fruits.tail()

,Fruit
Date,
2021-03-27,orange
2021-03-28,pineapple
2021-03-29,pears
2021-03-30,peach
2021-03-31,pears


In [15]:
# What is the distribution of fruits
fruits.Fruit.value_counts()

strawberry    15
apple         11
kiwi          10
pineapple     10
orange        10
watermelon     9
banana         7
grapes         7
pears          6
peach          5
Name: Fruit, dtype: int64

Clearly, strawberry is your favorite fruit. What if I wanted to see what fruit I ate the most in February? 

In [16]:
# Filter fruits by February 2021
fruits.filter(like='2021-02', axis=0)

,Fruit
Date,
2021-02-01,kiwi
2021-02-02,watermelon
2021-02-03,strawberry
2021-02-04,peach
2021-02-05,kiwi
2021-02-06,apple
2021-02-07,strawberry
2021-02-08,peach
2021-02-09,watermelon


This essentially checks the index (axis=0) so all rows to see if '2021-02' is in the Dates. This is a quick way to get your data when your index is a string.

What if I wanted to get all the dates where I ate strawberries? 

In [17]:
# Filter for strawberries
fruits[fruits.Fruit == 'strawberry']

,Fruit
Date,
2021-01-03,strawberry
2021-01-14,strawberry
2021-01-22,strawberry
2021-01-29,strawberry
2021-02-03,strawberry
2021-02-07,strawberry
2021-02-12,strawberry
2021-02-16,strawberry
2021-02-18,strawberry


What if I wanted to check for strawberries and apples?

There are two methods you can use, you can do this with a OR in your mask or you can use `df.isin` which takes a list and outputs when the data has values in that list.

In [18]:
# Get the mask to check for apples and strawberries
mask = fruits.Fruit.isin(['apple', 'strawberry'])

# View mask
mask.head()

Date
2021-01-01     True
2021-01-02    False
2021-01-03     True
2021-01-04    False
2021-01-05    False
Name: Fruit, dtype: bool

A mask basically gives you a True or False for each row of your DataFrame depending on if the condition holds true or not.

In [19]:
# All the dates you ate apples or strawberries
fruits[mask]

,Fruit
Date,
2021-01-01,apple
2021-01-03,strawberry
2021-01-14,strawberry
2021-01-15,apple
2021-01-17,apple
2021-01-22,strawberry
2021-01-27,apple
2021-01-29,strawberry
2021-01-30,apple


## Concatenation of Dataframes

If you have data stored in different sources (eg. in two different excel sheets), you would want to first clean the data first and then concatenate the data together for further usage.

We will show an example of how to join data from an excel file named "daily_activity.csv" and another file named "gym.csv"

In [20]:
# read in daily activity csv
activity = pd.read_csv('daily_activity.csv', index_col='Date')

# View our data
activity.head()

,Walk,Swim,Running
Date,,,
2021-01-01,89,36,26
2021-01-02,78,39,29
2021-01-03,68,39,27
2021-01-04,93,30,26
2021-01-05,68,26,26


In [21]:
# View the columns
activity.columns

Index(['Walk', 'Swim', 'Running'], dtype='object')

Now, let's say that we are a workout enthusiast and we also have the time to work out in the gym but that isn't stored in this dataframe. We read in another dataframe called `gym.csv` that stores the time we spend in the gym.

In [22]:
# Read in gym csv
gym = pd.read_csv('gym.csv')

# View our data
gym.head()

,Unnamed: 0,Date,Time
0,0,2021-01-01,27
1,1,2021-01-02,30
2,2,2021-01-03,27
3,3,2021-01-04,25
4,4,2021-01-05,26


In [23]:
# Select the needed columns
gym = gym[['Date', 'Time']]

# Set the index
gym.set_index('Date', inplace=True)

gym.head()

,Time
Date,
2021-01-01,27
2021-01-02,30
2021-01-03,27
2021-01-04,25
2021-01-05,26


Now we want all of our data in one table, so we need to join the two tables together. To do this, we need to have the same indices so that we can do an inner join on the index, which basically means that we are matching our data on the dates that both tables have. Hence, if any date does not appear in one of the tables, it would not show up in our inner join. 

To do this, we need to set our index to Date so that it matches the activity table. Then we use `pd.concat([df1, df2])` to join where we specify the parameter `join=inner` to show it is in an inner join and `axis=1` to show we are adding another column into our new table.

In [24]:
# Join the tables
daily_activity = pd.concat([activity, gym], join='inner', axis=1)

# View new data
daily_activity.head()

,Walk,Swim,Running,Time
Date,,,,
2021-01-01,89,36,26,27
2021-01-02,78,39,29,30
2021-01-03,68,39,27,27
2021-01-04,93,30,26,25
2021-01-05,68,26,26,26


In [25]:
# Check the length of your dataframes to ensure they are the same
len(daily_activity), len(activity), len(gym)

(120, 120, 120)

We can see that we have a new column called Time from our Gym table and that we had all the same dates between the two tables so they matched up. Next, `Time` makes sense as a column when it is just a gym table, but with this new table, we want to rename our columns so that it is `Gym` instead. 

In [26]:
# Rename columns
daily_activity.columns = ['Walk', 'Swim', 'Running', 'Gym']
daily_activity.columns

Index(['Walk', 'Swim', 'Running', 'Gym'], dtype='object')

You simply pass in a list of names to rename the columns by setting `df.columns`. In this case, your list of column names needs to match the number of columns in your table.

### Data summary


Lastly, we want to create a new DataFrame that summarizes the average number of minutes I walked, swam, ran and went to the gym for this time period. We will do this by creating a dictionary with our new values.

In [27]:
# Create a dictionary with the means
activity_dic = {'Walk': np.mean(daily_activity.Walk),
                'Running': daily_activity.Running.mean(),
                'Swim': np.mean(daily_activity.Swim),
                'Gym': np.mean(daily_activity.Gym)}
activity_dic

{'Walk': 76.04166666666667,
 'Running': 27.458333333333332,
 'Swim': 32.391666666666666,
 'Gym': 25.333333333333332}

In [28]:
# Create a DataFrame from the dictionary
activity_stats = pd.DataFrame(activity_dic, index=[0])
activity_stats

,Walk,Running,Swim,Gym
0,76.041667,27.458333,32.391667,25.333333


In [29]:
# Take the transpose of your data 
activity_stats = activity_stats.T

In [30]:
# Rename column
activity_stats.columns = ['mean']
activity_stats

,mean
Walk,76.041667
Running,27.458333
Swim,32.391667
Gym,25.333333


This was mainly to show you that sometimes you need to transpose your data when creating a DataFrame from a dictionary and that basically means swapping your rows and columns. 

A faster way to get the means for each activity type would be to do the following:

In [31]:
daily_activity.describe()

,Walk,Swim,Running,Gym
count,120.000000,120.000000,120.000000,120.000000
mean,76.041667,32.391667,27.458333,25.333333
std,11.310288,4.660392,1.689791,3.123060
min,55.000000,25.000000,25.000000,20.000000
25%,66.750000,28.000000,26.000000,23.000000
50%,77.000000,33.000000,27.000000,25.000000
75%,86.000000,36.250000,29.000000,28.000000
max,95.000000,40.000000,30.000000,30.000000


In [32]:
# Alternative way to get the mean
pd.DataFrame(daily_activity.describe().loc['mean'])

,mean
Walk,76.041667
Swim,32.391667
Running,27.458333
Gym,25.333333


### Data Types

Lastly, we want to look into how to convert datatypes, specifically dates.

In [33]:
# Check our datatypes for stock history
import yfinance as yf

ticker = yf.Ticker('META')

hist = ticker.history(start='2022-01-01', end='2022-08-01', interval='1mo')

hist.reset_index().dtypes

Date            datetime64[ns]
Open                   float64
High                   float64
Low                    float64
Close                  float64
Volume                   int64
Dividends                int64
Stock Splits             int64
dtype: object

In [34]:
# Check our datatypes for daily activity
daily_activity.reset_index().dtypes

Date       object
Walk        int64
Swim        int64
Running     int64
Gym         int64
dtype: object

You'll see that for our dates with Yahoo Finance we have datetimes; however, with our data from the csv, we have dates as an object. What if we wanted to convert our dates to datetime? 

This can be done very easily with DataFrames!

In [35]:
# Convert date index to datetime
daily_activity.index = pd.to_datetime(daily_activity.index)

In [36]:
# Check our new datatypes for daily activity
daily_activity.reset_index().dtypes

Date       datetime64[ns]
Walk                int64
Swim                int64
Running             int64
Gym                 int64
dtype: object

Remember to check data types when you are trying to join data together using either concat or inner joins. For example, when we joined our daily activity and gym data together, we were able to since both Date indices were an `object`; however, if one was an `object` while the other was a `datetime`, we would have to convert it.

For graphing purposes, datetime is more ideal.